## Torch Sparse - PyTorch Reference Api v2.9.0

In PyTorch, i singoli elementi dei tensori, modellati attraverso l'astrazione Tensor, vengono disposti contiguamente all'interno della memoria fisica. Questo porta ad implementazioni più efficienti di tutti quegli algoritmi che manipolano array e che richiedano un accesso rapido ai loro elementi.
Ora, nel caso della sparsificazione di una rete neurale, i tensori dei pesi sono costituiti da molti elementi nulli il cui numero dipende dal grado di sparsificazione. Per motivi legati all'ottimizzazione delle prestazioni, PyTorch supporta diversi formati per memorizzare tensori sparsificati che differiscono l'uno dall'altro per il modo in cui gli zeri sono distribuiti all'interno del tensore. Tutti i formati puntano a comprimere i dati al fine di ottenere un risparmio sia in memoria sia nelle computazioni eseguite su CPU o GPU. Nel caso in cui la sparsità fosse elevata oppure fosse altamente strutturata, l'impatto sulle prestazioni può essere significativo. Quindi possiamo guardare ai diversi formati di rappresentazione di tensori sparsi come ad una ottimizzazione delle prestazioni. Tuttavia non sempre queste rappresentazioni sparse portano a dei benefici e può anzi capitare che le prestazioni subiscano un degrado a seguito del loro impiego.

PyTorch supporta i seguenti formati ottenibili usando apposite funzioni a partire da tensori densi:
+ COO (Coordinate Format)
+ CSR (Compressed Sparse Row)
+ CSC (Compressed Sparse Column)
+ BSR (Block Compressed Sparse Row)
+ BSC (Block Compressed Sparse Column)
Il modo in cui vengono rappresentati i tensori sparsi, influenza le prestazioni delle operazioni fra di essi senza alcuna influenza sulla semantica dei dati

### Tensori sparsi semi-strutturati
La sparsificazione semi-strutturata è un formato per dati sparsificati introdotto da NVIDIA. E' anche nota come sparsificazione strutturata a granularità fine o come sparisificazione strutturata di tipo 2:4. In pratica per ogni blocco di 2n elementi al più n dovranno essere non nulli. Solo gli elementi non nulli verranno memorizzati. In PyTorch, la sparsità semi-strutturata viene implementata usando una sottoclasse di Tensor. Questo consente di eseguire funzioni efficienti per l'esecuzione di moltiplicazioni fra matrici. I tensori sparsi semi strutturati possono essere rappresentati usando un appropriato formato compresso che consente anche un risparmio in termini di spazio di memoria. Il formato compresso considera soltanto i valori non nulli accompagnati da alcuni metadati (detti anche maschera o mask). Il tutto viene rappresentato attraverso un singolo tensore piatto (ordine 1) e compresso che cioè contiene tutti e soli gli elementi non nulli l'uno di seguito all'altro subito seguiti dai metadati così da formare un unico blocco all'interno della memoria fisica. Ovvero
$$
    tensore compresso = [elementi non nulli | maschera metadati]
$$
Quindi se un tensore denso avesse dimensioni RxC il corrispondente tensore sparso in forma 2:4 sarebbe rappresentato da un tensore Rx(C/2) costituito dai soli valori non nulli ed un tensore Rx(C/2) di metadati rappresentati usando 2 bit per ciascuno.
Sia RxC la dimensione di un tensore e sia b il numero di bit utilizzati per rappresentare i singoli valori. Possiamo quantificare l'impatto sulla memoria
$$
    M_{dense} = R \times C \times b
$$
$$
    M_{sparse} = M_{specified} + M_{metadata} = (R \times \frac{C}{2} \times b) + (R \times \frac{C}{2} \times 2) = RCb(\frac{1}{2} + \frac{1}{b})
$$
Si può definire il rapporto di compressione come
$$
    C = \frac{M_{sparse}}{M_{dense}} = \frac{RCb(\frac{1}{2} + \frac{1}{b})}{RCb} = (\frac{1}{2} + \frac{1}{b})
$$
Se $b = 16$ otteniamo $C = 0.5625$ (compressione pari al 56.25%) mentre se $b = 8$ avremo $C = 0.625$ (compressione pari al 62.5%) 
